# Model Prediction
This notebook implements prediction using sliding window approach.

## Imports and Setup

In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Location

In [2]:
data_dir = "../data/"
output_dir = "../outputs/"
input_file = data_dir + '20200409_dataset.csv'

pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'
sentinel_dir = data_dir + 'sentinel2/'
false_pos_dir = data_dir + 'false_pos/'
indices_dir = data_dir + 'indices/'
tmp_dir = data_dir + 'tmp/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

areas = ['arauca', 'uribia', 'riohacha']

## Load Dataset

In [3]:
data = pd.read_csv(input_file).reset_index(drop=True)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (690163, 112)


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,savi_2020,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,target,area
0,0.15970,0.13735,0.1531,0.18700,0.2090,0.26320,0.30515,0.26965,0.3327,0.0411,...,0.187614,-0.509745,0.105128,0.239614,0.449106,-0.718433,-0.042537,0.901237,3,0
1,0.15970,0.13905,0.1454,0.17845,0.2090,0.26320,0.30515,0.26395,0.3327,0.0411,...,0.177058,-0.507485,0.105128,0.247826,0.464498,-0.716955,-0.039760,0.911490,3,0
2,0.16675,0.14875,0.1589,0.18605,0.2258,0.27945,0.32070,0.28085,0.3452,0.0416,...,0.179191,-0.524371,0.073259,0.262348,0.446475,-0.722188,-0.033995,0.875915,3,0


## Resample Dataset

In [4]:
la_guajira = data['area'].apply(lambda x: model_utils.get_la_guajira(x)) 
data.insert(len(data.columns)-2, 'la_guajira', la_guajira)

data = model_utils.resample(data, num_neg_samples=50000, false_pos_samples=5000, random_state=SEED)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (223439, 113)


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,la_guajira,target,area
0,0.1809,0.17790,0.1972,0.2371,0.2427,0.3009,0.33460,0.2913,0.3655,0.04435,...,-0.388713,0.040390,0.386829,0.574649,-0.603260,-0.010778,0.810930,1,4,0
1,0.1684,0.13690,0.1425,0.1458,0.1686,0.2696,0.30675,0.2652,0.3501,0.03810,...,-0.602441,0.098756,0.199285,0.318486,-0.752879,-0.057824,0.809911,1,4,0
2,0.1635,0.13655,0.1327,0.1346,0.1311,0.1754,0.20490,0.2100,0.2202,0.03560,...,-0.506141,0.128252,0.180842,0.403433,-0.754468,-0.048452,0.950226,1,4,0


## Machine Learning Pipeline

In [5]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
SEED = 42

## Define Features and Target Label

In [6]:
# Get target and feature columns
label = 'target'
features = [column for column in data.columns[:-2]]

# Convert target labels to binary
data[label] = data[label].replace({2:0, 3:0, 4:0})

print('Data dimensions: {}'.format(data.shape))
print('Class distribution:\n{}'.format(data['target'].value_counts()))

Data dimensions: (223439, 113)
Class distribution:
0    203792
1     19647
Name: target, dtype: int64


## Define Best Feature Set

In [7]:
lsvc_best_features = ['B5_2016', 'B10_2016', 'B1_2017', 'B2_2017', 'B5_2017', 'B10_2017', 'B6_2019', 'B7_2019', 'B9_2019', 'mbi_2019', 'B1_2020']

## Train Model

In [8]:
lsvc = LinearSVC(C=1000, random_state=SEED)

X = data[lsvc_best_features].fillna(0)
y = data[label]

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

lsvc.fit(X, y);

## Prediction: Memory-intensive Initial Approach

In [9]:
#pd.set_option('use_inf_as_na', True)
#area_dict = geoutils.get_filepaths(areas, sentinel_dir, indices_dir, pos_mask_dir, neg_mask_dir)
#data = geoutils.read_bands(area_dict, 'uribia')
#print('Data dimensions: {}'.format(data.shape))
#data.head(3)

In [10]:
#preds = lsvc._predict_proba_lr(data[lsvc_best_features].fillna(0))[:, 1]
#preds[(data.sum(axis=1) == 0)] = -1

#geoutils.save_predictions(
#    preds, 
#    image_src=area_dict['uribia']['images_cropped'][0], 
#    output_file=data_dir+'uribia_initial.tiff'
#)

## Prediction: Sliding Window Approach

In [ ]:
area_dict = geoutils.get_filepaths(areas, sentinel_dir, pos_mask_dir, neg_mask_dir, false_pos_dir)
grid_blocks = 9

for area in areas:
    output = output_dir + '20200410_{}.tif'.format(area)
    geoutils.get_preds_windowing(
        area=area, 
        area_dict=area_dict,
        model=lsvc, 
        tmp_dir=tmp_dir,
        best_features=lsvc_best_features, 
        scaler=scaler, 
        output=output, 
        grid_blocks=grid_blocks,
        threshold=0.01
    )

In [ ]:
#for area in areas:
#    filename = output_dir + '20200331_{}_thresh60.tif'.format(area)
#    !gsutil -q cp {filename} gs://immap-output/